# Nieve bays implementation on simple dataset

In [5]:
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import operator
from math import sqrt, exp
from matplotlib import pyplot as plt
import os
#read in file and then give it feature headers
os.path.isfile('/Users/16786/Desktop/wine.data')

df= pd.read_csv('/Users/16786/Desktop/wine.data')
df.columns = [ 'Alcohol', 'Malic acid', 'Ash', 'alcalinity of ash', 'Magnesium', 'Total phenols', 'Flavanoids', 'Nonflavanoid phenols',
'Proanthocyanins', 'olor intensity', 'Hue', 'OD280', 'Proline', 'isTrain']
#df.columns = ['sepal length', 'sepal width', ' petal length', 'petal width', 'FlowerClass']
#seperate aproximently 25% out to be in the test set with which sample goes in each set being random
df['isTrain'] = np.random.uniform(0,1,len(df)) <= .75
df.head(100)

,Alcohol,Malic acid,Ash,alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,olor intensity,Hue,OD280,Proline,isTrain
0,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,True
1,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,False
2,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,True
3,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,False
4,1,14.20,1.76,2.45,15.2,112,3.27,3.39,0.34,1.97,6.75,1.05,2.85,False
5,1,14.39,1.87,2.45,14.6,96,2.50,2.52,0.30,1.98,5.25,1.02,3.58,True
6,1,14.06,2.15,2.61,17.6,121,2.60,2.51,0.31,1.25,5.05,1.06,3.58,True
7,1,14.83,1.64,2.17,14.0,97,2.80,2.98,0.29,1.98,5.20,1.08,2.85,True
8,1,13.86,1.35,2.27,16.0,98,2.98,3.15,0.22,1.85,7.22,1.01,3.55,True
9,1,14.10,2.16,2.30,18.0,105,2.95,3.32,0.22,2.38,5.75,1.25,3.17,True


In [6]:
#adjusting the alcohol classes to start at 0 instead of 1.
for x in range (0,len(df)):
    if df.iloc[x,0] == 1: 
        df.iloc[x,0] = 0
    if df.iloc[x,0] == 2: 
        df.iloc[x,0] = 1
    if df.iloc[x,0] == 3: 
        df.iloc[x,0] = 2
df.head(10)
features = df.columns[1:13]
#seperating train and test into two different dataframes for time effecency 
train, test = df[df['isTrain']==True], df[df['isTrain']==False]
#lengths of each set this will change with every run
print('length of train set ' + str(len(train)))
print('length of test set '+ str(len(test)))
print((train[0:0+1]).iloc[0,4])
df.head(10)
#extracting mean, SD and variance of each columns contents based on its flowerclass for the P(Xi|y) claculation later 
meanlist0 = np.mean(df.loc[df['Alcohol'] == 0])
meanlist1 = np.mean(df.loc[df['Alcohol'] == 1])
meanlist2 = np.mean(df.loc[df['Alcohol'] == 2])
meanlistAll = [meanlist0,meanlist1,meanlist2]
SDlist0 = np.std(df.loc[df['Alcohol'] == 0])
SDlist1 = np.std(df.loc[df['Alcohol'] == 1])
SDlist2 = np.std(df.loc[df['Alcohol'] == 2])
SDlistAll = [SDlist0,SDlist1,SDlist2]
varlist0 = SDlist0 * SDlist0
varlist1 = SDlist1 * SDlist1
varlist2 = SDlist2 * SDlist2

varlistAll = [varlist0,varlist1,varlist2]
       

length of train set 138
length of test set 39
11.2


In [7]:
#populate list of classifications
test_set_classifications = [0]
for n in range (0, len(test)-1):
        test_set_classifications.append(-1)
#preforms a classification for each member in the test set and store the results in test_set_classifications        
for x in range (0, len(test)-1):
    productlist = np.ones(3)
    #tests P(Xi|C)(P(C))/P(X) for each class C in the dataset and stores the resulting propablitys in the list productlist
    for i in range(0, 3):
        product = 1
        #loop to calculate P(Xi|C) for each feature in the dataset and store the product in product
        for j in range(1, len(features)):
            #calculating term before the exponent in the Gaussian Naive Bayes P(Xi|C) equation
            component1 = 1/sqrt(2*3.14*varlistAll[i][j])
            #calculating the exponent in the Gaussian Naive Bayes P(Xi|C) equation
            component2 = exp(-0.5 * pow(((test[x:x+1].iloc[0,j]) - meanlistAll[i][j]),2)/varlistAll[i][j])
            product = product * component1 * component2
        #multiplying by the prior probobility    
        product = product * (len(train['Alcohol'] == i)/len(train))
        productlist[i] = product
    
    #finds the index of the maximum value in the probability list then store that index in a list containing the classification of each test set sample the index represents class
    test_set_classifications[x] = np.argmax(productlist)


In [8]:

correct = 0
print(test[0:1].iloc[0,0])
print(test_set_classifications[0])
#computes the accuracy of the model
for x in range (0, len(test)):
    a = 0
    if test_set_classifications[x] == test[x:x+1].iloc[0,0]:
        correct += 1
   # print(correct)
accuracy = correct/len(test)
print("The accuracy is ")
print(accuracy)

0
0
The accuracy is 
0.9743589743589743
